<a href="https://colab.research.google.com/github/Raniamea/arabic-video-summarisation/blob/main/VideoProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎥 Arabic Video Multimodal Validator and Summarizer

This Colab notebook lets you input the name of an Arabic video file and automatically performs:
- Audio transcription (Arabic)
- Scene/keyframe caption validation using Sentence-BERT and CLIP
- (Optional) Abstractive summarization with mBART

---

In [ ]:
# 1. Setup Environment
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q transformers sentence-transformers torchaudio opencv-python Pillow
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define base paths
base_path = "/content/drive/MyDrive/ArabicVideoSummariser"
videos_path = os.path.join(base_path, "videos")
transcripts_path = os.path.join(base_path, "transcripts")
captions_path = os.path.join(base_path, "captions")
keyframes_path = os.path.join(base_path, "keyframes")
os.makedirs(transcripts_path, exist_ok=True)

In [ ]:
# 2. Input Video Filename
video_filename = input("Enter the name of the video file (e.g., MyVideo.mp4): ")
video_path = os.path.join(videos_path, video_filename)
assert os.path.exists(video_path), f"Video file not found: {video_path}"

## 🔊 3. Transcribe Arabic Audio using Whisper

In [ ]:
# Transcribe the video
import whisper
model = whisper.load_model("large")
result = model.transcribe(video_path, language="ar")

# Save transcription
transcript_txt = os.path.join(transcripts_path, f"{os.path.splitext(video_filename)[0]}_ar.txt")
with open(transcript_txt, "w", encoding="utf-8") as f:
    f.write(result['text'])

## 🧠 4. Process Transcript into Overlapping Chunks

In [ ]:
# Load transcript and split into chunks
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

with open(transcript_txt, encoding='utf-8') as f:
    full_transcript = f.read()

words = word_tokenize(full_transcript)
chunk_size = 30
step = 15
transcript_chunks = [' '.join(words[i:i+chunk_size]) for i in range(0, len(words)-chunk_size, step)]

## 🖼️ 5. Load Scene Captions

In [ ]:
# Load captions from JSON
import json
captions_json = os.path.join(captions_path, f"{os.path.splitext(video_filename)[0]}.json")
with open(captions_json, encoding='utf-8') as f:
    scenes = json.load(f)
scene_captions = [(scene, data["arabic"]) for scene, data in scenes.items()]

## 🔡 6. Embed Captions and Transcript Chunks

In [ ]:
# Encode using multilingual Sentence-BERT
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

caption_texts = [text for _, text in scene_captions]
caption_embeddings = model.encode(caption_texts, convert_to_tensor=True)
transcript_embeddings = model.encode(transcript_chunks, convert_to_tensor=True)

## 🔗 7. Match Captions to Transcript Chunks

In [ ]:
# Find best transcript match for each caption
results = []
similarities = util.cos_sim(caption_embeddings, transcript_embeddings)
for i, (scene_id, caption_text) in enumerate(scene_captions):
    sim_scores = similarities[i]
    top_idx = sim_scores.argmax().item()
    results.append({
        "scene_id": scene_id,
        "caption": caption_text,
        "best_transcript_chunk": transcript_chunks[top_idx],
        "similarity_score": float(sim_scores[top_idx])
    })

## 📥 8. Output Results

In [ ]:
# Display a few matches
import pandas as pd
df = pd.DataFrame(results)
df[['scene_id', 'caption', 'best_transcript_chunk', 'similarity_score']].head(10)